In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os

In [84]:
# og_path = r"C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\materials\\xl_97"
og_path = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\"
end_path = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_10\\"
export = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\export\price_data.xlsx"

In [3]:
#Convert xl 97 format to xl 10 
def convert_97_to_10(og, end):
    for item in os.listdir(og):
        full_path = os.path.join(og, item)
        print(full_path)
        name = item.split('.')[0]
        new_name = name + '.xlsx'
        print(new_name)
        p.save_book_as(file_name=(full_path),dest_file_name = end + new_name)

# returns job and address from df 
def job_local(df):
    job = df.iloc[4,3]
    locat = df.iloc[8, 3]
    return job, locat
    

# def 

In [75]:
#covert
convert_97_to_10(og_path, end_path)

C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 01.xls
Vendor RFQ Analysis 01.xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 02.xls
Vendor RFQ Analysis 02.xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 03.xls
Vendor RFQ Analysis 03.xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 04 .xls
Vendor RFQ Analysis 04 .xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 05.xls
Vendor RFQ Analysis 05.xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 2.

In [96]:
list_of_scopes = ['Aggregate', 'Asphalt', 'Concrete', 'Manholes', 'RCP Pipe', 'Water Works']
ag_dfs = []
asp_dfs = []
conc_dfs = []
man_dfs = []
rcp_dfs = []
wat_dfs = []
full_melt = []
for item in os.listdir(end_path):
    full_path = os.path.join(end_path, item)
    xl = pd.read_excel(full_path)
    df = pd.DataFrame(xl)
    job, location = job_local(df)
    # add job and location column 
    df['Job'] = job
    df['Address'] = location
    
    
    # job/location

    #remove nas for first row/ reset index / add date column 
    no_na_prod = df[df['Belair Sitework Services'].notna()]
    no_na_prod.reset_index()
    date = df.iloc[len(df)-1, 0]
    #rename columns
    no_na_prod = no_na_prod.rename(columns={'Belair Sitework Services': 'Product','Unnamed: 1': 'Sub', 'Unnamed: 5': 'Scope',
                                            'Unnamed: 8': 'QTY','Unnamed: 10': 'Unit'
                            , 'Unnamed: 4': 'Date','Unnamed: 13': 'Q1','Unnamed: 16': 'Q2','Unnamed: 19': 'Q3','Unnamed: 22': 'Q4' })
    no_na_prod['Date'] = date
    # remove unnecesary columns 
    reduced = no_na_prod.drop(df.columns[[2,6,7,9,11,12,14,15,17,18,20,21,23]], axis=1)
    reduced = reduced.reset_index().head(60)


    #
    reference_row_index = reduced.index[reduced['Product'] == 'Vendor Information'].tolist()

    #
    long_list = reference_row_index.copy()
    long_list.append(len(reduced))

    # fill the scope column with relevent scope 
    for ref_ind in range(len(reference_row_index)):
        scope = reduced.iloc[(reference_row_index[ref_ind]-1), 3]
        reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
    # remove more unnecesary columns 
    reduced = reduced.drop(reduced.columns[[0,3]], axis=1)
    reduced
    # drop rows with blank scope field
    no_na_scope = reduced[reduced['Scope'].notna()]
    # remove all rows without prices 
    no_na_price = no_na_scope[no_na_scope['Q1'].notna()]
    #renove characters
    melted = pd.melt(no_na_price, id_vars=['Product', 'Scope'], value_vars=['Q1', 'Q2', 'Q3', 'Q4'])
    melted['value'] = melted['value'].replace('[^.0-9\-]', '', regex=True)

    melted = melted[melted['value'] != '']
    melted['value'] = melted['value'].astype(float)
    melted = melted[melted['value'] > 0]
    full_melt.append(melted)
    for scp in list_of_scopes:
        scp_df = melted[melted['Scope'] == scp]
        if scp == 'Aggregate':
            ag_dfs.append(scp_df)
        elif scp == 'Asphalt':
            asp_dfs.append(scp_df)
        elif scp == 'Concrete':
            conc_dfs.append(scp_df)
        elif scp == 'Manholes':
            man_dfs.append(scp_df)
        elif scp == 'RCP Pipe':
            rcp_dfs.append(scp_df)           
        elif scp == 'Water Works':
            wat_dfs.append(scp_df)              
            


C:\Users\willd\AppData\Local\Temp\ipykernel_14912\347076466.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_14912\347076466.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_14912\347076466.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [97]:
full = pd.concat(full_melt)
full

,Product,Scope,variable,value
1,Class 1 Structural Fill,Aggregate,Q1,12.39
2,#57/67,Aggregate,Q1,32.92
3,CDOT Class 6 Road Base,Aggregate,Q1,27.78
4,Bedding Squeegee,Aggregate,Q1,32.92
5,ASTM #8 Stone,Aggregate,Q1,32.92
...,...,...,...,...
29,"21"" RCP CL-III",RCP Pipe,Q2,65.29
30,"24"" RCP CL-III",RCP Pipe,Q2,71.81
31,"30"" RCP CL-III",RCP Pipe,Q2,99.02
49,Water Stop Gaskets,Water Works,Q3,12.75


In [98]:
ag = pd.concat(ag_dfs)
asp = pd.concat(asp_dfs)
conc = pd.concat(conc_dfs)
man = pd.concat(man_dfs)
rcp = pd.concat(rcp_dfs)
wat = pd.concat(wat_dfs)
# asp_dfs
# conc_dfs = []
# man_dfs = []
# rcp_dfs = []
# wat_dfs = []

In [100]:
wat_dfs[1]

,Product,Scope,variable,value
18,"02"" Corp Stop Comp",Water Works,Q1,444.05


In [82]:
group = ag.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
group

,Product,mean,max,count
0,#57/67,40.91,40.91,6
1,"6"" VTC Rock",41.46,41.46,6
2,Bedding Squeegee,27.47,27.47,6
3,Bedding Type II,40.91,40.91,6
4,CDOT Class 6 Road Base,24.05,24.05,6
5,"Type M Rip Rap (d/50=12"")",42.23,42.23,6


In [101]:
ag_group = ag.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
asp_group = asp.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
conc_group = conc.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
man_group = man.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
rcp_group = rcp.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()
wat_group = wat.groupby('Product')['value'].agg(['mean', 'max', 'count']).reset_index()

In [102]:
writer = pd.ExcelWriter(export, engine="xlsxwriter")
ag_group.to_excel(writer, sheet_name="Aggregate")
asp_group.to_excel(writer, sheet_name="Asphalt")
conc_group.to_excel(writer, sheet_name="Concrete")
man_group.to_excel(writer, sheet_name="Manhole")
rcp_group.to_excel(writer, sheet_name="RCP")
wat_group.to_excel(writer, sheet_name="Water works")
writer.close()

In [5]:
# job/location

#remove nas for first row/ reset index / add date column 
no_na_prod = df[df['Belair Sitework Services'].notna()]
no_na_prod.reset_index()
date = df.iloc[len(df)-1, 0]
#rename columns
no_na_prod = no_na_prod.rename(columns={'Belair Sitework Services': 'Product','Unnamed: 1': 'Sub', 'Unnamed: 5': 'Scope',
                                        'Unnamed: 8': 'QTY','Unnamed: 10': 'Unit'
                        , 'Unnamed: 4': 'Date','Unnamed: 13': 'Q1','Unnamed: 16': 'Q2','Unnamed: 19': 'Q3','Unnamed: 22': 'Q4' })
no_na_prod['Date'] = date
# remove unnecesary columns 
reduced = no_na_prod.drop(df.columns[[2,6,7,9,11,12,14,15,17,18,20,21,23]], axis=1)
reduced = reduced.reset_index().head(60)


#
reference_row_index = reduced.index[reduced['Product'] == 'Vendor Information'].tolist()

#
long_list = reference_row_index.copy()
long_list.append(len(reduced))

# fill the scope column with relevent scope 
for ref_ind in range(len(reference_row_index)):
    scope = reduced.iloc[(reference_row_index[ref_ind]-1), 3]
    reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
# remove more unnecesary columns 
reduced = reduced.drop(reduced.columns[[0,3]], axis=1)
reduced
# drop rows with blank scope field
no_na_scope = reduced[reduced['Scope'].notna()]
# remove all rows without prices 
no_na_price = no_na_scope[no_na_scope['Q1'].notna()]
#renove characters
melted['value'] = melted['value'].replace('[^.0-9\-]', '', regex=True)

melted = melted[melted['value'] != '']
melted['value'] = melted['value'].astype(float)
melted = melted[melted['value'] > 0]



C:\Users\willd\AppData\Local\Temp\ipykernel_14912\12263653.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope


In [6]:
no_na_scope

,Product,Sub,Date,Scope,QTY,Unit,Q1,Q2,Q3,Q4,Job,Address
5,Job Number:,NaN,12/15/2023 12:50:55 PM,Bid Number:,001,NaN,Billing Address:,6950 S. Jordan Rd,NaN,NaN,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
11,Vendor Information,NaN,12/15/2023 12:50:55 PM,Aggregate,NaN,NaN,NaN,NaN,NaN,NaN,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
12,Quote,Company Name,12/15/2023 12:50:55 PM,Aggregate,NaN,NaN,NaN,Phone,NaN,Minority,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
13,1,Martin Marietta,12/15/2023 12:50:55 PM,Aggregate,NaN,NaN,NaN,720-245-6412,NaN,NaN,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
14,2,"Albert Frei & Sons, Inc",12/15/2023 12:50:55 PM,Aggregate,NaN,NaN,NaN,303.289.1837,NaN,NaN,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
15,Material Prices,NaN,12/15/2023 12:50:55 PM,Aggregate,NaN,NaN,NaN,NaN,NaN,NaN,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
16,"6"" VTC Rock",NaN,12/15/2023 12:50:55 PM,Aggregate,151.80,TON,$41.46,$0.00,$0.00,$0.00,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
17,CDOT Class 6 Road Base,NaN,12/15/2023 12:50:55 PM,Aggregate,370.00,TON,$24.05,$0.00,$0.00,$0.00,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
18,#57/67,NaN,12/15/2023 12:50:55 PM,Aggregate,"1,733.87",TON,$40.91,$0.00,$0.00,$0.00,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"
19,Bedding Type II,NaN,12/15/2023 12:50:55 PM,Aggregate,89.21,TON,$40.91,$0.00,$0.00,$0.00,Fox Park Phase 2 EU RFP#001,"W 44th & N Huron St, Denver, CO"


In [60]:
melted = pd.melt(no_na_price, id_vars=['Product', 'Scope'], value_vars=['Q1', 'Q2', 'Q3', 'Q4'])
melted

,Product,Scope,variable,value
0,Job Number:,Bid Number:,Q1,Billing Address:
1,"6"" VTC Rock",Aggregate,Q1,$41.46
2,CDOT Class 6 Road Base,Aggregate,Q1,$24.05
3,#57/67,Aggregate,Q1,$40.91
4,Bedding Type II,Aggregate,Q1,$40.91
...,...,...,...,...
63,"21"" RCP CL-III",RCP Pipe,Q4,$0.00
64,"24"" RCP CL-III",RCP Pipe,Q4,$0.00
65,"30"" RCP CL-III",RCP Pipe,Q4,$0.00
66,Water Stop Gaskets,Water Works,Q4,$0.00


In [61]:
melted['value'] = melted['value'].replace('[^.0-9\-]', '', regex=True)

In [65]:
melted

,Product,Scope,variable,value
0,Job Number:,Bid Number:,Q1,
1,"6"" VTC Rock",Aggregate,Q1,41.46
2,CDOT Class 6 Road Base,Aggregate,Q1,24.05
3,#57/67,Aggregate,Q1,40.91
4,Bedding Type II,Aggregate,Q1,40.91
...,...,...,...,...
63,"21"" RCP CL-III",RCP Pipe,Q4,0.00
64,"24"" RCP CL-III",RCP Pipe,Q4,0.00
65,"30"" RCP CL-III",RCP Pipe,Q4,0.00
66,Water Stop Gaskets,Water Works,Q4,0.00


In [66]:
melted = melted[melted['value'] != '']
melted['value'] = melted['value'].astype(float)
melted = melted[melted['value'] > 0]

C:\Users\willd\AppData\Local\Temp\ipykernel_14912\4031788192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted['value'] = melted['value'].astype(float)


In [67]:
melted

,Product,Scope,variable,value
1,"6"" VTC Rock",Aggregate,Q1,41.46
2,CDOT Class 6 Road Base,Aggregate,Q1,24.05
3,#57/67,Aggregate,Q1,40.91
4,Bedding Type II,Aggregate,Q1,40.91
5,"Type M Rip Rap (d/50=12"")",Aggregate,Q1,42.23
6,Bedding Squeegee,Aggregate,Q1,27.47
7,Concrete,Concrete,Q1,217.62
8,Thrust Block Utilities,Concrete,Q1,87.05
9,"60"" Precast Manhole",Manholes,Q1,3568.97
10,"48"" Precast Manhole",Manholes,Q1,3416.63


In [48]:
melted['amount'] = str(melted['value'].replace('$', '').replace(',', ''))

In [49]:
melted = melted[melted[]]

,Product,Scope,variable,value,amount
0,Job Number:,Bid Number:,Q1,Billing Address:,0 Billing Address:\n1 $41.46...
1,"6"" VTC Rock",Aggregate,Q1,$41.46,0 Billing Address:\n1 $41.46...
2,CDOT Class 6 Road Base,Aggregate,Q1,$24.05,0 Billing Address:\n1 $41.46...
3,#57/67,Aggregate,Q1,$40.91,0 Billing Address:\n1 $41.46...
4,Bedding Type II,Aggregate,Q1,$40.91,0 Billing Address:\n1 $41.46...
...,...,...,...,...,...
63,"21"" RCP CL-III",RCP Pipe,Q4,$0.00,0 Billing Address:\n1 $41.46...
64,"24"" RCP CL-III",RCP Pipe,Q4,$0.00,0 Billing Address:\n1 $41.46...
65,"30"" RCP CL-III",RCP Pipe,Q4,$0.00,0 Billing Address:\n1 $41.46...
66,Water Stop Gaskets,Water Works,Q4,$0.00,0 Billing Address:\n1 $41.46...


In [25]:
for scp in list_of_scopes:
    scp_df = no_na_scope[no_na_scope['Scope'] == scp]
    new_name_index = scp_df.index[scp_df['Product'] == 'Material Prices'].tolist()
    if '1' in scp_df['Product']:
#         vend1 = scp_df.loc[scp_df['Product']=='1', 'Sub']
        vend1 = scp_df.at['1', 'Sub']
    else: 
        vent1 = 'QT1'
    
#     vend1 = scp_df.query("Product=='1'")['Sub'].values[]
    print(vend1)
    

13    Martin Marietta
Name: Sub, dtype: object
13    Martin Marietta
Name: Sub, dtype: object
13    Martin Marietta
Name: Sub, dtype: object
13    Martin Marietta
Name: Sub, dtype: object
13    Martin Marietta
Name: Sub, dtype: object
13    Martin Marietta
Name: Sub, dtype: object


In [20]:
# new reference index 
new_row_index = no_na_scope.index[no_na_scope['Product'] == 'Vendor Information'].tolist()

In [22]:
new_name_index = no_na_scope.index[no_na_scope['Product'] == 'Material Prices'].tolist()

In [23]:
new_name_index

[15, 27, 36, 45, 57]

In [ ]:
for scp in blah:
    

In [13]:
for ref_mum in reference_row_index:
#     print(df.iloc[scope, 3])
    print(ref_mum)
    print(df.iloc[(ref_mum-1), 3], df.iloc[(ref_mum+2), 1], df.iloc[(ref_mum+3), 1], df.iloc[(ref_mum+4), 1])

NameError: name 'reference_row_index' is not defined

In [24]:
for scope in scope_list:
    print(df.iloc[scope, 3])
    print(df.iloc[, 3])
    

Aggregate
Concrete
Manholes
RCP Pipe
Water Works
Water Works (continued)


In [77]:
df = df[df['Unnamed: 0'].notna()]
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Scope
0,7630 Dahlia Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Phone:,303-394-1300,NaN,NaN,NaN,NaN,NaN,
1,"Commerce City, CO 80022",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Fax:,303-394-1301,NaN,NaN,NaN,NaN,NaN,
2,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Vendor RFQ Analysis Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Project Name:,NaN,NaN,Fox Park Phase 2 EU RFP#001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Saunders Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,"00.75"" Insulator",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,...,NaN,$0.00,NaN,NaN,$52.78,NaN,NaN,$0.00,NaN,Water Works
283,"00.75"" Meter Setter Flare",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,...,NaN,$0.00,NaN,NaN,$97.93,NaN,NaN,$0.00,NaN,Water Works
284,"06"" Solid Sleeve L/A",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,...,NaN,$0.00,NaN,NaN,$146.62,NaN,NaN,$0.00,NaN,Water Works
285,Mirafi 140 N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,031.06",NaN,...,NaN,$0.00,NaN,NaN,$1.46,NaN,NaN,$0.00,NaN,Water Works


In [84]:
list1 = df['Scope'].unique()

AttributeError: 'numpy.ndarray' object has no attribute 'remove'

In [78]:
df.head(60)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Scope
0,7630 Dahlia Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Phone:,303-394-1300,NaN,NaN,NaN,NaN,NaN,
1,"Commerce City, CO 80022",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Fax:,303-394-1301,NaN,NaN,NaN,NaN,NaN,
2,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Vendor RFQ Analysis Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Project Name:,NaN,NaN,Fox Park Phase 2 EU RFP#001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Saunders Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,Job Number:,NaN,NaN,NaN,NaN,Bid Number:,NaN,NaN,001,NaN,...,NaN,6950 S. Jordan Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,Bid As:,NaN,NaN,Sub Contractor,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Centenial, CO 80112 USA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,Estimator:,NaN,NaN,EW Will Davis UT Sara Locke,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,(303) 699-9000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,Project Address:,NaN,NaN,"W 44th & N Huron St, Denver, CO",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Lance Christopher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,Completion Date:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [51]:
list_of_df = []
for item in os.listdir(end_path):
    full_path = os.path.join(end_path, item)
    job, date, location = job_details(full_path)
    print(job)
    print(date)
    print(location)
    
    

Fox Park Phase 2 EU RFP#001
None
W 44th & N Huron St, Denver, CO


In [53]:
pd.read_excel(full_path).head(50)

,Belair Sitework Services,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,7630 Dahlia Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Phone:,303-394-1300,NaN,NaN,NaN,NaN,NaN
1,"Commerce City, CO 80022",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Fax:,303-394-1301,NaN,NaN,NaN,NaN,NaN
2,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vendor RFQ Analysis Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Project Name:,NaN,NaN,Fox Park Phase 2 EU RFP#001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Saunders Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Job Number:,NaN,NaN,NaN,NaN,Bid Number:,NaN,NaN,001,NaN,...,NaN,NaN,6950 S. Jordan Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bid As:,NaN,NaN,Sub Contractor,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"Centenial, CO 80112 USA",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estimator:,NaN,NaN,EW Will Davis UT Sara Locke,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,(303) 699-9000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Project Address:,NaN,NaN,"W 44th & N Huron St, Denver, CO",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Lance Christopher,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Completion Date:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
